## Load the dataset

In [1]:
# Load the dataset
import pandas as pd

df = pd.read_csv("C:\\Users\\xnoor\\OneDrive - Lambton College\\Documents\\GitHub\\smart-price-api\\datasets\\transformed_ebay_data.csv")

In [2]:
# Filtered out the category that have less than 1000 listings
threshold = 1000  
category_counts = df['remainder__Category'].value_counts()
filtered_categories = category_counts[category_counts > threshold].index
filtered_df = df[df['remainder__Category'].isin(filtered_categories)]

print(filtered_df['remainder__Category'].value_counts())

remainder__Category
Books                           19305
Greeting Cards & Invitations    19205
Wreaths, Garlands & Plants      10293
Contemporary Manufacture         8784
DVDs & Blu-ray Discs             7280
Nativity Items                   5831
Contemporary                     4311
Vintage & Antique                4231
Gift Cards                       3717
Other Games                      3614
Gift Tags & Stickers             2830
Plush Baby Toys                  1574
Satellite Dishes                 1405
Crystals                         1395
VHS Tapes                        1363
Blankets & Throws                1334
Playing Cards                    1222
Mugs                             1190
Masks & Eye Masks                1087
Name: count, dtype: int64


In [3]:
print(filtered_df.describe())


          Unnamed: 0    num__Price  num__Feedback Score  \
count   99971.000000  99971.000000         99971.000000   
mean    62836.178572     -0.014219             0.075423   
std     37830.031778      0.509933             1.109390   
min         0.000000     -0.085141            -0.303916   
25%     29307.500000     -0.063789            -0.303474   
50%     60781.000000     -0.046058            -0.301280   
75%     96152.500000     -0.015227            -0.284215   
max    127759.000000    151.767463             4.045962   

       cat__Listing Type_Auction  cat__Listing Type_AuctionWithBIN  \
count               99971.000000                      99971.000000   
mean                    0.021736                          0.003751   
std                     0.145822                          0.061131   
min                     0.000000                          0.000000   
25%                     0.000000                          0.000000   
50%                     0.000000                

### Find Max, Median, Mean End Price for each category

In [4]:
# Calculate median, max, and percentiles for each category
category_price_stats = filtered_df.groupby('remainder__Category')['remainder__Price_in_USD'].agg(
    max_price='max',
    median_price='median',
    price_25th_percentile=lambda x: x.quantile(0.25),
    price_75th_percentile=lambda x: x.quantile(0.75)
).reset_index()

print(category_price_stats)

             remainder__Category  max_price  median_price  \
0              Blankets & Throws     198.00        18.445   
1                          Books    2222.00         8.310   
2                   Contemporary     295.00        10.000   
3       Contemporary Manufacture    4900.00        16.880   
4                       Crystals     299.00         0.130   
5           DVDs & Blu-ray Discs    1150.00        11.490   
6                     Gift Cards     499.99        22.990   
7           Gift Tags & Stickers     300.00         8.950   
8   Greeting Cards & Invitations   49500.00         8.890   
9              Masks & Eye Masks     921.78        13.670   
10                          Mugs     175.05        15.950   
11                Nativity Items    3191.99        29.990   
12                   Other Games     702.00        14.700   
13                 Playing Cards     198.00        14.290   
14               Plush Baby Toys     144.27        14.990   
15              Satellit

In [5]:
# Merge the calculated statistics back into the main DataFrame
filtered_df = filtered_df.merge(category_price_stats, on='remainder__Category', how='left')

### Remove Non-Predictive Columns

In [6]:
# Define columns to exclude (non-predictive columns)
columns_to_exclude = ['Unnamed: 0', 'remainder__Title', 'remainder__Item ID', 
                      'remainder__Currency', 'remainder__Seller', 'remainder__Category',
                      'remainder__Gallery URL', 'remainder__Large Image URL', 
                      'remainder__Super Size Image URL', 'remainder__View Item URL', 'remainder__Shipping Cost', 
                      'remainder__Marketplace', 'remainder__Start Time', 'remainder__End Time',
                      'remainder__Store Name', 'remainder__Store URL']

### Define Price Columns

In [7]:
# Define the important price columns (targets and additional price statistics)
price_columns = ['num__Price', 'max_price', 'median_price', 'price_25th_percentile', 'price_75th_percentile', 'remainder__Price_in_USD']


### 1. Define Features and Target Variable

In [8]:
# Prepare the feature set by excluding non-predictive and price columns
X = filtered_df.drop(columns=columns_to_exclude + price_columns)

# Set the target variable(s) (e.g., median, percentiles, or actual price if you’re predicting that)
# Assuming here you're predicting price range as lower (25th) and upper (75th) percentiles
y = filtered_df[['price_25th_percentile', 'price_75th_percentile']]

### 2. Split the Data for Training

In [9]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Ensure price columns are not in X_train or X_test
print(X_train.columns)

print(X_train.isna().sum())

Index(['num__Feedback Score', 'cat__Listing Type_Auction',
       'cat__Listing Type_AuctionWithBIN', 'cat__Listing Type_FixedPrice',
       'cat__Listing Type_StoreInventory', 'cat__Shipping Type_Calculated',
       'cat__Shipping Type_CalculatedDomesticFlatInternational',
       'cat__Shipping Type_Flat',
       'cat__Shipping Type_FlatDomesticCalculatedInternational',
       'cat__Shipping Type_Free', 'cat__Shipping Type_FreePickup',
       'cat__Shipping Type_Freight', 'cat__Shipping Type_NotSpecified',
       'cat__Top Rated Seller_False', 'cat__Top Rated Seller_True',
       'cat__Condition_Acceptable', 'cat__Condition_Brand New',
       'cat__Condition_Certified - Refurbished',
       'cat__Condition_For parts or not working', 'cat__Condition_Good',
       'cat__Condition_Like New', 'cat__Condition_New',
       'cat__Condition_New other (see details)', 'cat__Condition_New with box',
       'cat__Condition_New with defects', 'cat__Condition_New with tags',
       'cat__Condition_

## Random Forest Regressor

We'll use a Random Forest Regressor to predict the lower and upper bounds (25th and 75th percentiles) of the price range.

### 3. Train the Model

In [29]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the Random Forest model for multi-target regression
model = RandomForestRegressor(n_estimators=200, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

### 4. Make Predictions

In [30]:
# Make predictions on the test data
y_pred = model.predict(X_test)

### 5. Evaluate the Model

We’ll use Mean Absolute Error (MAE) and Mean Squared Error (MSE) to evaluate the model’s performance for both the lower and upper bounds of the price range.


In [32]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Separate the predictions for the lower and upper bounds
predicted_lower_bound = y_pred[:, 0]
predicted_upper_bound = y_pred[:, 1]

# Separate the actual values for the lower and upper bounds in the test set
actual_lower_bound = y_test.iloc[:, 0]
actual_upper_bound = y_test.iloc[:, 1]

# Calculate MAE and MSE for the lower bound
mae_lower = mean_absolute_error(actual_lower_bound, predicted_lower_bound)
mse_lower = mean_squared_error(actual_lower_bound, predicted_lower_bound)

# Calculate MAE and MSE for the upper bound
mae_upper = mean_absolute_error(actual_upper_bound, predicted_upper_bound)
mse_upper = mean_squared_error(actual_upper_bound, predicted_upper_bound)

print("Mean Absolute Error for Lower Bound:", mae_lower)
print("Mean Squared Error for Lower Bound:", mse_lower)
print("Mean Absolute Error for Upper Bound:", mae_upper)
print("Mean Squared Error for Upper Bound:", mse_upper)

from sklearn.metrics import r2_score

# Calculate R-squared for the lower bound (25th percentile)
r2_lower = r2_score(actual_lower_bound, predicted_lower_bound)

# Calculate R-squared for the upper bound (75th percentile)
r2_upper = r2_score(actual_upper_bound, predicted_upper_bound)

print("R-squared for Lower Bound:", r2_lower)
print("R-squared for Upper Bound:", r2_upper)

Mean Absolute Error for Lower Bound: 1.3074860307037222
Mean Squared Error for Lower Bound: 6.621299373680201
Mean Absolute Error for Upper Bound: 4.585271456475267
Mean Squared Error for Upper Bound: 90.73717558036276
R-squared for Lower Bound: 0.7721477087819235
R-squared for Upper Bound: 0.7404418588760983


**Summary**

**Modeling Choice:**

A RandomForestRegressor model is trained initially for multi-target regression, aiming to predict both the 25th and 75th price percentiles.

**Performance Metrics:**

R-squared for Lower Bound (25th Percentile): Approximately 0.77
R-squared for Upper Bound (75th Percentile): Approximately 0.74
These R-squared values indicate that the model explains around 75% of the variance in the lower and upper price bounds, suggesting strong predictive performance relative to previous attempts.

**Observations:**

With an R-squared near 0.75, this model provides a reliable estimate of price range bounds (25th and 75th percentiles), which could be practically useful for guiding pricing strategies.
However, the relatively high MAE for the upper bound indicates room for further tuning or additional features to refine predictions in high price ranges.